In [ ]:
import random
from collections import defaultdict
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import seaborn as sns
import torch
import torch.utils.data
from sklearn.linear_model import LogisticRegression
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE
from sklearn.mixture import GaussianMixture
from torch.utils.data import TensorDataset

from datasets.crime import CrimeDataset
from generative.gmm import GMM
from real_nvp_encoder import FlowEncoder

In [ ]:
sns.set_theme()

In [ ]:
PROJECT_ROOT = Path('.').absolute().parent

In [ ]:
dataset = 'crime'
gamma = 1.0
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
alpha = 0.05
# lr = 1e-2
# weight_decay = 1e-4
# kl_start = 0
# kl_end = 50
# protected_att = None
n_blocks = 4
batch_size = 128
# dec_epochs = 100
# prior_epochs = 150
# n_epochs = 60
# adv_epochs = 60
# prior = 'gmm'
gmm_comps1 = 4
gmm_comps2 = 2
# gamma = 1.0
# n_flows = 1
seed = 100
# train_dec = False
# log_epochs = 10
p_test = 0.2
p_val = 0.2
# with_test = False
# fair_criterion = 'stat_parity'

In [ ]:
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)


In [ ]:
plots_dir = PROJECT_ROOT / 'plots' / dataset
plots_dir.mkdir(parents=True, exist_ok=True)
model_dir = PROJECT_ROOT / 'code' / dataset / f'gamma_{gamma}'

In [ ]:
train_dataset = CrimeDataset('train', p_test=p_test, p_val=p_val)
valid_dataset = CrimeDataset('validation', p_test=p_test, p_val=p_val)
test_dataset = CrimeDataset('test', p_test=p_test, p_val=p_val)

In [ ]:
feats = np.array([3, 15, 42, 43, 44, 49])
column_ids = [
    col for col, idx in train_dataset.column_ids.items() if idx in feats
]

In [ ]:
train_all = train_dataset.features[:, feats]
valid_all = valid_dataset.features[:, feats]
test_all = test_dataset.features[:, feats]

train_prot = train_dataset.protected
valid_prot = valid_dataset.protected
test_prot = test_dataset.protected

train_targets = train_dataset.labels
valid_targets = valid_dataset.labels
test_targets = test_dataset.labels

In [ ]:
lb, ub = dict(), dict()

for idx, feature_idx in enumerate(feats):
    lb[feature_idx] = torch.min(train_all[:, idx])
    ub[feature_idx] = torch.max(train_all[:, idx])

In [ ]:
def normalize(x, a, b):
    return 0.5 + (1 - alpha) * ((x - a) / (b - a) - 0.5)

def denormalize(z, a, b):
    return ((z - 0.5) / (1 - alpha) + 0.5) * (b - a) + a

def normalize_data(data):
    for idx, feature_idx in enumerate(feats):
        data[:, idx] = normalize(data[:, idx], lb[feature_idx], ub[feature_idx])
    return data

def denormalize_data(data):
    for idx, feature_idx in enumerate(feats):
        data[:, idx] = denormalize(data[:, idx], lb[feature_idx], ub[feature_idx])
    return data

In [ ]:
train_all = normalize_data(train_all)
valid_all = normalize_data(valid_all)
test_all = normalize_data(test_all)

In [ ]:
train1, train2 = train_all[train_prot == 1], train_all[train_prot == 0]
targets1, targets2 = train_targets[train_prot == 1].long(), train_targets[train_prot == 0].long()
train1_loader = torch.utils.data.DataLoader(TensorDataset(train1, targets1), batch_size=batch_size, shuffle=True)
train2_loader = torch.utils.data.DataLoader(TensorDataset(train2, targets2), batch_size=batch_size, shuffle=True)

valid1, valid2 = valid_all[valid_prot == 1], valid_all[valid_prot == 0]
v_targets1, v_targets2 = valid_targets[valid_prot == 1].long(), valid_targets[valid_prot == 0].long()
valid1_loader = torch.utils.data.DataLoader(TensorDataset(valid1, v_targets1), batch_size=8, shuffle=True)
valid2_loader = torch.utils.data.DataLoader(TensorDataset(valid2, v_targets2), batch_size=8, shuffle=True)

test1, test2 = test_all[test_prot == 1], test_all[test_prot == 0]
t_targets1, t_targets2 = test_targets[test_prot == 1].long(), test_targets[test_prot == 0].long()
test1_loader = torch.utils.data.DataLoader(TensorDataset(test1, t_targets1), batch_size=8, shuffle=True)
test2_loader = torch.utils.data.DataLoader(TensorDataset(test2, t_targets2), batch_size=8, shuffle=True)

In [ ]:
gaussian_mixture1 = GaussianMixture(n_components=gmm_comps1, n_init=1, covariance_type='full')
gaussian_mixture2 = GaussianMixture(n_components=gmm_comps2, n_init=1, covariance_type='full')

gaussian_mixture1.weights_ = np.load(model_dir / 'prior1_weights.npy')
gaussian_mixture1.means_ = np.load(model_dir / 'prior1_means.npy')
gaussian_mixture1.covariances_ = np.load(model_dir / 'prior1_covs.npy')

gaussian_mixture2.weights_ = np.load(model_dir / 'prior2_weights.npy')
gaussian_mixture2.means_ = np.load(model_dir / 'prior2_means.npy')
gaussian_mixture2.covariances_ = np.load(model_dir / 'prior2_covs.npy')

prior1 = GMM(gaussian_mixture1, device=device)
prior2 = GMM(gaussian_mixture2, device=device)

In [ ]:
masks = []
for i in range(20):
    t = np.array([j % 2 for j in range(feats.shape[0])])
    np.random.shuffle(t)
    masks += [t, 1 - t]

flow1 = FlowEncoder(None, feats.shape[0], [50, 50], n_blocks, masks).to(device)
flow2 = FlowEncoder(None, feats.shape[0], [50, 50], n_blocks, masks).to(device)

flow1.load_state_dict(torch.load(model_dir / 'flow1.pt'))
flow2.load_state_dict(torch.load(model_dir / 'flow2.pt'))

In [ ]:
mappings = defaultdict(list)

for (x1, y1), (x2, y2) in zip(train1_loader, train2_loader):

    # clamp has no effect on train data
    x1 = torch.clamp(x1, alpha / 2, 1 - alpha).logit()
    x2 = torch.clamp(x2, alpha / 2, 1 - alpha).logit()


    x1_z1, _ = flow1.inverse(x1)
    x1_x2, _ = flow2.forward(x1_z1)

    mappings['x1_real'].append(x1.sigmoid())
    mappings['x2_fake'].append(x1_x2.sigmoid())
    mappings['x1_real_logp'].append(prior1.log_prob(x1))
    mappings['x2_fake_logp'].append(prior2.log_prob(x1_x2))
    mappings['z1'].append(x1_z1)
    mappings['y1'].append(y1)

    x2_z2, _ = flow2.inverse(x2)
    x2_x1, _ = flow1.forward(x2_z2)

    mappings['x1_fake'].append(x2_x1.sigmoid())
    mappings['x2_real'].append(x2.sigmoid())
    mappings['x1_fake_logp'].append(prior1.log_prob(x2_x1))
    mappings['x2_real_logp'].append(prior2.log_prob(x2))
    mappings['z2'].append(x2_z2)
    mappings['y2'].append(y2)

In [ ]:
# undo flow normalization
x1_real = denormalize_data(torch.vstack(mappings['x1_real']))
x2_real = denormalize_data(torch.vstack(mappings['x2_real']))
x1_fake = denormalize_data(torch.vstack(mappings['x1_fake']))
x2_fake = denormalize_data(torch.vstack(mappings['x2_fake']))

# undo preprocessing normalization
x1_real_denormalized = train_dataset.std[feats] * x1_real + train_dataset.mean[feats]
x2_real_denormalized = train_dataset.std[feats] * x2_real + train_dataset.mean[feats]
x1_fake_denormalized = train_dataset.std[feats] * x1_fake + train_dataset.mean[feats]
x2_fake_denormalized = train_dataset.std[feats] * x2_fake + train_dataset.mean[feats]

x1_real = x1_real.cpu().detach()
x2_real = x2_real.cpu().detach()
x1_fake = x1_fake.cpu().detach()
x2_fake = x2_fake.cpu().detach()

x1_real_denormalized = x1_real_denormalized.cpu().detach()
x2_real_denormalized = x2_real_denormalized.cpu().detach()
x1_fake_denormalized = x1_fake_denormalized.cpu().detach()
x2_fake_denormalized = x2_fake_denormalized.cpu().detach()

x1_fake_logp = torch.cat(mappings['x1_fake_logp']).cpu().detach()
x2_fake_logp = torch.cat(mappings['x2_fake_logp']).cpu().detach()
x1_real_logp = torch.cat(mappings['x1_real_logp']).cpu().detach()
x2_real_logp = torch.cat(mappings['x2_real_logp']).cpu().detach()

z1 = torch.vstack(mappings['z1']).cpu().detach()
z2 = torch.vstack(mappings['z2']).cpu().detach()
y1 = torch.cat(mappings['y1']).cpu().detach()
y2 = torch.cat(mappings['y2']).cpu().detach()

In [ ]:
x1 = torch.cat((x1_real, x1_fake))
x2 = torch.cat((x2_fake, x2_real))
x1_denormalized = torch.cat((x1_real_denormalized, x1_fake_denormalized))
x2_denormalized = torch.cat((x2_fake_denormalized, x2_real_denormalized))

x1_logp = torch.cat((x1_real_logp, x1_fake_logp))
x2_logp = torch.cat((x2_fake_logp, x2_real_logp))

z = torch.cat((z1, z2))
y = torch.cat((y1, y2))

n_x1 = x1_real.shape[0]
n_x2 = x2_real.shape[0]

In [ ]:
n_clusters = 4
kmeans_x1 = KMeans(n_clusters=n_clusters, random_state=0).fit(x1)
kmeans_x2 = KMeans(n_clusters=n_clusters, random_state=0).fit(x2)

perplexity = 35
x1_t_sne = TSNE(perplexity=perplexity, random_state=75 if gamma == 0 else 56).fit_transform(x1)
x2_t_sne = TSNE(perplexity=perplexity, random_state=75 if gamma == 0 else 56).fit_transform(x2)

# rescale to [-1, 1]
l1, u1 = x1_t_sne.min(0), x1_t_sne.max(0)
l2, u2 = x2_t_sne.min(0), x2_t_sne.max(0)

x1_t_sne = (2 * x1_t_sne - (u1 + l1)) / (u1 - l1)
x2_t_sne = (2 * x2_t_sne - (u2 + l2)) / (u2 - l2)

fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(8, 4))

ax[0].scatter(x1_t_sne[:n_x1, 0], x1_t_sne[:n_x1, 1], c=kmeans_x1.labels_[:n_x1], marker='.', cmap='tab10', label='real')
ax[0].scatter(x1_t_sne[n_x1:, 0], x1_t_sne[n_x1:, 1], c=kmeans_x1 .labels_[n_x1:], marker='x', cmap='tab10', label='matched', s=20)
ax[1].scatter(x2_t_sne[n_x1:, 0], x2_t_sne[n_x1:, 1], c=kmeans_x1.labels_[n_x1:], marker='.', cmap='tab10', label='real')
ax[1].scatter(x2_t_sne[:n_x1, 0], x2_t_sne[:n_x1, 1], c=kmeans_x1 .labels_[:n_x1], marker='x', cmap='tab10', label='matched', s=20)

for idx in range(2):
    ax[idx].set_xticklabels(list())
    ax[idx].set_yticklabels(list())
    ax[idx].legend()
    ax[idx].get_legend().legendHandles[0].set_color('k')
    ax[idx].get_legend().legendHandles[1].set_color('k')
    ax[idx].set_xlim(left=-1.1, right=1.1)
    ax[idx].set_ylim(bottom=-1.1, top=1.1)

ax[0].set_title(r'Non-White (a = 1)')
ax[1].set_title(r'White $(a = 0)$')

fig.transFigure.inverted()
ax0tr = ax[0].transData
ax1tr = ax[1].transData
figtr = fig.transFigure.inverted()

ptA = figtr.transform(ax0tr.transform((1.15, -0.1)))
ptB = figtr.transform(ax1tr.transform((-1.15, -0.1)))
arrow = patches.FancyArrowPatch(
    ptA, ptB, transform=fig.transFigure, connectionstyle="arc3,rad=0.5",
    arrowstyle='simple, head_width=5, head_length=5', color='k'
)
fig.patches.append(arrow)

fig.text(0.495, 0.495, 'FNF')

ptA = figtr.transform(ax1tr.transform((-1.15, 0.15)))
ptB = figtr.transform(ax0tr.transform((1.15, 0.15)))
arrow = patches.FancyArrowPatch(
    ptA, ptB, transform=fig.transFigure, connectionstyle="arc3,rad=0.5",
    arrowstyle='simple, head_width=5, head_length=5', color='k'
)
fig.patches.append(arrow)

plt.savefig(
    plots_dir / f'gamma_{gamma}_perplexity_{perplexity}_n_clusters_{n_clusters}.eps',
    bbox_inches='tight'
)

In [ ]:
clusters_x1 = pd.DataFrame(columns=['logp'] + column_ids)
clusters_x2 = pd.DataFrame(columns=['logp'] + column_ids)

for cluster in range(n_clusters):
    clusters_x1.loc[cluster] = torch.cat((
        x1_logp[kmeans_x1.labels_ == cluster].mean().unsqueeze(0),
        x1_denormalized[kmeans_x1.labels_ == cluster].mean(axis=0)
    )).numpy()
    clusters_x2.loc[cluster] = torch.cat((
        x2_logp[kmeans_x1.labels_ == cluster].mean().unsqueeze(0),
        x2_denormalized[kmeans_x1.labels_ == cluster].mean(axis=0)
    )).numpy()

clusters_x1.to_csv(
    plots_dir / f'gamma_{gamma}_n_clusters_{n_clusters}_x1.csv', index=False
)
clusters_x2.to_csv(
    plots_dir / f'gamma_{gamma}_n_clusters_{n_clusters}_x2.csv', index=False
)

In [ ]:
train1 = denormalize_data(train1)
train2 = denormalize_data(train2)

print(train_dataset.std[feats] * train1.min(0)[0] + train_dataset.mean[feats])
print(train_dataset.std[feats] * train2.min(0)[0]+ train_dataset.mean[feats])
print()
print(train_dataset.std[feats] * train1.mean(0)+ train_dataset.mean[feats])
print(train_dataset.std[feats] * train2.mean(0)+ train_dataset.mean[feats])
print()
print(train_dataset.std[feats] * train1.max(0)[0]+ train_dataset.mean[feats])
print(train_dataset.std[feats] * train2.max(0)[0]+ train_dataset.mean[feats])

In [ ]:
clf = LogisticRegression(random_state=0).fit(z, y)
y_hat = clf.predict(z)
clf.score(z, y)

In [ ]:
mappings = defaultdict(list)

for x1_i, z_i in zip(x1[y_hat == 0], z[y_hat == 0]):
    z_i_nn = z[y_hat == 1][torch.norm(z_i - z[y_hat == 1], dim=1).argmin()]

    for beta in np.linspace(0, 1, 11):
        z_new = torch.unsqueeze(z_i + beta * (z_i_nn - z_i), 0)

        if clf.predict(z_new):
            break

    x1_i_new, _ = flow1.forward(z_new.to(device))

    mappings['x1_old'].append(x1_i.to(device))
    mappings['x1_new'].append(x1_i_new.sigmoid().to(device))

for x2_i, z_i in zip(x2[y_hat == 0], z[y_hat == 0]):
    z_i_nn = z[y_hat == 1][torch.norm(z_i - z[y_hat == 1], dim=1).argmin()]

    for beta in np.linspace(0, 1, 11):
        z_new = torch.unsqueeze(z_i + beta * (z_i_nn - z_i), 0)

        if clf.predict(z_new):
            break

    x2_i_new, _ = flow2.forward(z_new.to(device))

    mappings['x2_old'].append(x2_i.to(device))
    mappings['x2_new'].append(x2_i_new.sigmoid().to(device))

In [ ]:
x1_old = torch.vstack(mappings['x1_old'])
x2_old = torch.vstack(mappings['x2_old'])

# undo flow normalization
x1_new = denormalize_data(torch.vstack(mappings['x1_new']))
x2_new = denormalize_data(torch.vstack(mappings['x2_new']))

# undo preprocessing normalization
x1_old = train_dataset.std[feats] * x1_old + train_dataset.mean[feats]
x1_new = train_dataset.std[feats] * x1_new + train_dataset.mean[feats]
x2_old = train_dataset.std[feats] * x2_old + train_dataset.mean[feats]
x2_new = train_dataset.std[feats] * x2_new + train_dataset.mean[feats]

In [ ]:
avg_recourse = pd.DataFrame(
    [torch.mean(x1_new - x1_old, 0).cpu().detach().numpy(),
     torch.mean(x2_new - x2_old, 0).cpu().detach().numpy()],
    columns=column_ids, index=['Non-White', 'White']
)
avg_recourse.to_csv(plots_dir / f'recourse_gamma_{gamma}.csv')